In [4]:
import re
import torch
from torch.utils.data import DataLoader, Dataset
import random
import numpy as np
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
import platform
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
from sklearn.model_selection import KFold

# Dropout
Dropout을 적용하고 싶은 레이어 전에 Dropout을 넣어줘야 함<br>
train시에는 꺼지지 않은 값이 2배가 됨

In [6]:
x = torch.Tensor([[1,2,3,4], [4,5,6,7]])
nn.Dropout(0.5).forward(x)

tensor([[ 2.,  0.,  6.,  8.],
        [ 0., 10., 12.,  0.]])

# nn.CrossEntropyLoss()
logsoftmax + nllloss이기 때문에 모델 안에 softmax레이어를 넣을 필요 없음<br>
target은 long이어야 하고 input은 데이터 개수 x output_dim(=카테고리개수)여야 함<br>
output은 데이터개수 차원이고 안의 내용은 카테고리 숫자여야함 <br>

In [17]:
>>> loss = nn.CrossEntropyLoss()
>>> input = torch.randn(3, 5, requires_grad=True)
>>> target = torch.empty(3, dtype=torch.long).random_(5)
>>> output = loss(input, target)
>>> output.backward()

In [16]:
target.shape, input.shape

(torch.Size([3]), torch.Size([3, 5]))

# inplace operation

In [25]:
x.add_(1)
x

tensor([[ 4.,  5.,  6.,  7.],
        [ 7.,  8.,  9., 10.]])

In [ ]:
74